In [34]:
# Basic imports
import numpy as np
import pandas as pd

# Plotting tools
import bokeh
from bokeh.io import show

import cellseg.plot

bokeh.io.output_notebook()

Loading BokehJS ...

## Figure plotting
The purpose of this code is to generate the figure panels for publication from the data collected by `in_vitro_cell_segmentation.ipynb`

We can start by importing the data that we want to work with:

In [35]:
# Use read_csv to create a dataframe from the data csv 
df_data = pd.read_csv('data/quantification/20220927-data.csv', comment='#')

# Use read_csv to create a dataframe from the look up table
df_meta = pd.read_csv('data/quantification/20220927_data_lookup_table.csv', comment='#')

Then, we can merge the dataframes to connect the metadata with the measured data:

In [36]:
# Use merge the two dataframes, using round, plate, and well to combine them
df = pd.merge(df_data, df_meta, on=["Round", "Plate", "Well"])

We can sort and curate the resulting dataframe: 

In [37]:
#We can sort the values to ensure they're in a logical order
df = df.sort_values(by = ['Round', 'Plate', 'Well'], axis = 0)

# Drop any empty rows and reset the index so that it is in sequence
df = df.dropna().reset_index(drop=True)

We can perform some initial analysis on the data to determine the percent positive area of each well, the scaled area (to enable plotting at a smaller size), and the total brightness per signal area.

In [38]:
# Determine the percent positive and add it to a new column
df['Percent Positive'] = df['Signal Area']*100/df['Bright Field Area']
df['Percent Positive Scaled'] = df['Percent Positive']*0.5
df['Total Brightness per Signal Area'] = df['Total Brightness']/df['Signal Area']

We can then examine the resulting dataframe: 

In [39]:
# Create a index to only include valid data (selected by experimenter)
inds = df['Include'] == True

# Create a reduced data frame of this data
df_reduced = df.loc[inds]

# Look at the dataframe 
df_reduced.head()

,Date,Round,Plate,Well,Count,Cells Quantified,Brightness List,Bright Field Area,Signal Area,Total Brightness,...,90% Confidence Interval,Virus,Receptor,Include,Dose,DMSO (%),Brinzolamide (nM),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,2022-08-31 18:07:15.091087,1,1,1,293,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.0800825593395253, '2': 0.0833759590792...",166689,14092,3534.819608,...,"[0.064593837535014, 0.4457565667776543]",PHP.eB,Ly6a,True,1.000000e+09,0.0,0.0,8.454067,4.227034,0.250839
1,2022-08-31 18:07:16.537289,1,1,2,408,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.38899619549312264, '2': 0.093681917211...",152698,13466,3199.658824,...,"[0.06732026143790851, 0.43567486253760757]",PHP.eB,Ly6a,True,1.000000e+09,0.0,0.0,8.818714,4.409357,0.237610
2,2022-08-31 18:07:18.101663,1,1,3,501,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.07352941176470587, '2': 0.108650519031...",166536,15691,4430.505882,...,"[0.0784313725490196, 0.49664650907091806]",PHP.eB,Ly6a,True,1.000000e+09,0.0,0.0,9.421987,4.710993,0.282360
3,2022-08-31 18:07:19.502329,1,1,4,351,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.3840607210626185, '2': 0.1131907308377...",164404,12472,3075.960784,...,"[0.06686274509803922, 0.43730278436160797]",PHP.eB,Ly6a,True,5.000000e+08,0.0,0.0,7.586190,3.793095,0.246629
4,2022-08-31 18:07:20.939185,1,1,5,397,[ 1 2 3 4 5 6 7 8 9 10 11 ...,"{'1': 0.2784313725490196, '2': 0.1019607843137...",179028,13464,3490.862745,...,"[0.06845938375350141, 0.44557593077716917]",PHP.eB,Ly6a,True,5.000000e+08,0.0,0.0,7.520611,3.760306,0.259274


Now that we have our data, we can use it to generate the figures.

---

## Figure 2

### Part C

First, we want to compare PHP.eB against AAV9 at a range of doses without a receptor.

In [40]:
# We want to make a new dataframe with just the Ly6a and receptor-free conditions.
# First, we can find the indices of these receptors.
receptor_inds = (df_reduced['Receptor'] == 'Ly6a') | (df_reduced['Receptor'] == 'None')
virus_inds = (df_reduced['Virus'] == 'PHP.eB') | (df_reduced['Virus'] == 'AAV9')
round_inds = (df_reduced['Round'] == 10)

inds = receptor_inds & virus_inds & round_inds

# Make this a new dataframe df_f2c
df_f2c = df_reduced.loc[inds]

# We need to convert the dose values to a string to allow us to plot catagorically
df_f2c = df_f2c.astype({"Dose": str})

# We can determine the mean of each of the values
df_f2c = df_f2c.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [41]:
# Setting our index to only examine data with Receptor none, we can plot the untransfected AAV9 and PHP.eB transfections
inds = (df_f2c['Receptor'] == 'None')

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2c[inds], 
                                                                               'Virus', 
                                                                               'Dose', 
                                                                               y_values = ['500000000.0', '1000000000.0', '5000000000.0', '10000000000.0', '50000000000.0'], 
                                                                               plot_width = 350, 
                                                                               plot_height = 500)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 74.89 
The least positive value is: 3.14 
The brightest value is: 0.79 
The dimmest value is: 0.39


In [42]:
# Setting our index to only examine data with Receptor none, we can plot the untransfected AAV9 and PHP.eB transfections
inds = (df_f2c['Receptor'] == 'Ly6a')

p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2c[inds], 
                                                                               'Virus', 
                                                                               'Dose', 
                                                                               y_values = ['500000000.0', '1000000000.0', '5000000000.0', '10000000000.0', '50000000000.0'], 
                                                                               plot_width = 350, 
                                                                               plot_height = 500)

bokeh.io.show(p)

print("The most positive value is: %0.2f \nThe least positive value is: %0.2f \nThe brightest value is: %0.2f \nThe dimmest value is: %0.2f" % (max_pos, min_pos, max_brightness, min_brightness))

The most positive value is: 39.44 
The least positive value is: 3.99 
The brightest value is: 0.71 
The dimmest value is: 0.40


### Part D

In [44]:
dose_inds = (df_reduced['Dose'] == 1e9)

df_f2d = df_reduced.loc[dose_inds]

inds = (((df_f2d['Virus'] != 'PHP.C1') & (df_f2d['Virus'] != 'PHP.C3') & (df_f2d['Virus'] != '9P13') & (df_f2d['Virus'] != '9P33')) & (df_f2d['Round'] < 8)) | \
        ((df_f2d['Virus'] == 'PHP.C1') | (df_f2d['Virus'] == 'PHP.C3')) & (df_f2d['Round'] < 7)

df_f2d = df_f2d.loc[inds]

# We can determine the mean of each of the values
df_f2d = df_f2d.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [45]:
p, min_pos, max_pos, min_brightness, max_brightness = cellseg.plot.bubble_plot(df_f2d, 
                                                                               'Virus', 
                                                                               'Receptor',
                                                                               x_values = ['PHP.eB',
                                                                                           'CAP-B10',
                                                                                           'CAP-B22',
                                                                                           'PHP.N',
                                                                                           'PHP.C1',
                                                                                           'PHP.C2',
                                                                                           'PHP.C3',
                                                                                           'PHP.C4',
                                                                                           'AAV-F'],
                                                                               y_values = ['Tek',
                                                                                           'Slco1c1',
                                                                                           'Slco1a4',
                                                                                             'Slc7a1',
                                                                                             'Slc6a6',
                                                                                             'Slc2a1', 
                                                                                             'Slc22a8',
                                                                                             'Serinc3',
                                                                                             'Prom1',
                                                                                             'Podxl',
                                                                                             'Pecam1', 
                                                                                             'Paqr5', 
                                                                                             'Ocln', 
                                                                                             'Ly6e', 
                                                                                             'Ly6c1', 
                                                                                             'Ly6a', 
                                                                                             'Kitl', 
                                                                                             'Kdr', 
                                                                                             'Itm2b', 
                                                                                             'Igf1r', 
                                                                                             'Ifitm3', 
                                                                                             'Ifitm2', 
                                                                                             'Flt1', 
                                                                                             'Fcgrt', 
                                                                                             'Esam', 
                                                                                             'Eng',
                                                                                             'Clic4', 
                                                                                             'Clic1', 
                                                                                             'Clec2d', 
                                                                                             'Cldn5', 
                                                                                             'Cdh5', 
                                                                                             'Cd34', 
                                                                                             'Car4', 
                                                                                             'Bsg', 
                                                                                             'App', 
                                                                                             'Adgrl4',
                                                                                             'Adgrf5', 
                                                                                             'Acvrl1', 
                                                                                             'Abcg2', 
                                                                                             'Abcb1a'],
                                                                               plot_width = 600, 
                                                                               plot_height = 1200)

bokeh.io.show(p)

In [ ]:
source = ColumnDataSource(df_edf8)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F'],
                      #'9P31',
                      #'9P36'],
          y_range = ['Tek',
                     'Slco1c1',
                     'Slco1a4',
                     'Slc7a1',
                     'Slc6a6',
                     'Slc2a1', 
                     'Slc22a8',
                     'Serinc3',
                     'Prom1',
                     'Podxl',
                     'Pecam1', 
                     'Paqr5', 
                     'Ocln', 
                     'Ly6e', 
                     'Ly6c1', 
                     'Ly6a', 
                     'Kitl', 
                     'Kdr', 
                     'Itm2b', 
                     'Igf1r', 
                     'Ifitm3', 
                     'Ifitm2', 
                     'Flt1', 
                     'Fcgrt', 
                     'Esam', 
                     'Eng',
                     'Clic4', 
                     'Clic1', 
                     'Clec2d', 
                     'Cldn5', 
                     'Cdh5', 
                     'Cd34', 
                     'Car4', 
                     'Bsg', 
                     'App', 
                     'Adgrl4',
                     'Adgrf5', 
                     'Acvrl1', 
                     'Abcg2', 
                     'Abcb1a'],
           title = 'Candidate Receptors',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=1000)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf8['Total Brightness per Signal Area'].min(), high = df_edf8['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

## Extended Data Figure 5

In [ ]:
# We want to make a new dataframe with just the Ly6a and receptor-free conditions.
# First, we can find the indices of these receptors.
receptor_inds = (df_reduced['Receptor'] == 'Ly6a') | (df_reduced['Receptor'] == 'None')
virus_inds = (df_reduced['Virus'] == 'PHP.eB') | (df_reduced['Virus'] == 'AAV9')
round_inds = (df_reduced['Round'] == 10)
dose_inds = (df_reduced['Dose'] == 1e9)

inds = receptor_inds & virus_inds & round_inds

# Make this a new dataframe df_f2c
df_f2c = df_reduced.loc[inds]

# We need to convert the dose values to a string to allow us to plot catagorically
df_f2c = df_f2c.astype({"Dose": str})

# We can determine the mean of each of the values
df_f2c = df_f2c.groupby(['Virus', 'Receptor', 'Dose']).mean(numeric_only = True).reset_index()

In [ ]:
# We want to make a new dataframe with just the 1e9 dose.
# First, we can find the indices of these receptors.
inds = (df_reduced['Dose'] == 1e9)

# Make this a new dataframe df_edf5
df_edf5 = df_reduced.loc[inds]

In [43]:
# Next, we can reduce this to just to round 7.
inds = (((df_edf5['Virus'] != 'PHP.C1') & (df_edf5['Virus'] != 'PHP.C3') & (df_edf5['Round'] == 7)) | ((df_edf5['Virus'] == 'PHP.C1') | (df_edf5['Virus'] == 'PHP.C3')) & (df_edf5['Round'] < 8))

df_edf5 = df_edf5.loc[inds]

In [44]:
# We can determine the mean of each of the values
df_edf5 = df_edf5.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_14144\1495839934.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_edf5 = df_edf5.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [48]:
p = bubble_plot(df_edf5, 'Virus', 'Receptor')

bokeh.io.show(p)

In [45]:
source = ColumnDataSource(df_edf5)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      '9P08',
                      '9P16',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F',
                      '9P13',
                      '9P33',
                      '9P39'],
          y_range = ['Cd59',
                     'Lynx1',
                     'Lypd5',
                     'Ly6h',
                     'Ly6e', 
                     'Ly6c1',
                     'Ly6a'],
           title = 'Receptor screen of Ly6 family',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf5['Total Brightness per Signal Area'].min(), high = df_edf5['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [20]:
inds = (df_edf5['Virus'] != 'PHP.C1') & (df_edf5['Virus'] != 'PHP.C3')
min(df_edf5.loc[inds]['Percent Positive']), max(df_edf5.loc[inds]['Percent Positive']), df_edf5['Total Brightness per Signal Area'].min(), df_edf5['Total Brightness per Signal Area'].max()

(2.0251856581547347,
 47.733569217593974,
 0.06336345691653478,
 0.624552131784229)

## Extended Data Figure 6

In [21]:
# We want to make a new dataframe with just the 1e9 dose.
# First, we can find the indices of these receptors.
inds = (df_reduced['Dose'] == 1e9)

# Make this a new dataframe df_edf6
df_edf6 = df_reduced.loc[inds]

In [22]:
inds = (df_edf6['Round'] == 10)

df_edf6 = df_edf6.loc[inds]

In [23]:
# We can determine the mean of each of the values
df_edf6 = df_edf6.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\1032797064.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_edf6 = df_edf6.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [24]:
source = ColumnDataSource(df_edf6)

p = figure(x_range = ['9P31',
                      '9P36'],
          y_range = ['Car15',
                     'Car14',
                     'Car12',
                     'Car9',
                     'Car4'],
           title = 'Receptor screen of carbonic anhydrase family',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width = 400, 
           height = 500)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf6['Total Brightness per Signal Area'].min(), high = df_edf6['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [25]:
inds = (df_edf6['Receptor'] == 'Car4') | (df_edf6['Receptor'] == 'Car9') | (df_edf6['Receptor'] == 'Car12') | (df_edf6['Receptor'] == 'Car14') | (df_edf6['Receptor'] == 'Car15')
min(df_edf6.loc[inds]['Percent Positive']), max(df_edf6.loc[inds]['Percent Positive']), df_edf6['Total Brightness per Signal Area'].min(), df_edf6['Total Brightness per Signal Area'].max()

(5.224234670553813, 51.98639846511173, 0.34270870574499085, 0.785822957013865)

## Extended Data Figure 8

In [26]:
#We can sort the values to ensure they're in a logical order
inds = (df_reduced['Dose'] == 1e9)

df_edf8 = df_reduced.loc[inds]

In [27]:
# Next, we can reduce this to just to round 7.
inds = (((df_edf8['Virus'] != 'PHP.C1') & (df_edf8['Virus'] != 'PHP.C3') & (df_edf8['Virus'] != '9P13') & (df_edf8['Virus'] != '9P33')) & (df_edf8['Round'] < 8)) | \
        ((df_edf8['Virus'] == 'PHP.C1') | (df_edf8['Virus'] == 'PHP.C3')) & (df_edf8['Round'] < 7)

df_edf8 = df_edf8.loc[inds]

In [28]:
# We can determine the mean of each of the values
df_edf8 = df_edf8.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\3496360055.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_edf8 = df_edf8.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [29]:
source = ColumnDataSource(df_edf8)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F'],
                      #'9P31',
                      #'9P36'],
          y_range = ['Tek',
                     'Slco1c1',
                     'Slco1a4',
                     'Slc7a1',
                     'Slc6a6',
                     'Slc2a1', 
                     'Slc22a8',
                     'Serinc3',
                     'Prom1',
                     'Podxl',
                     'Pecam1', 
                     'Paqr5', 
                     'Ocln', 
                     'Ly6e', 
                     'Ly6c1', 
                     'Ly6a', 
                     'Kitl', 
                     'Kdr', 
                     'Itm2b', 
                     'Igf1r', 
                     'Ifitm3', 
                     'Ifitm2', 
                     'Flt1', 
                     'Fcgrt', 
                     'Esam', 
                     'Eng',
                     'Clic4', 
                     'Clic1', 
                     'Clec2d', 
                     'Cldn5', 
                     'Cdh5', 
                     'Cd34', 
                     'Car4', 
                     'Bsg', 
                     'App', 
                     'Adgrl4',
                     'Adgrf5', 
                     'Acvrl1', 
                     'Abcg2', 
                     'Abcb1a'],
           title = 'Candidate Receptors',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=1000)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf8['Total Brightness per Signal Area'].min(), high = df_edf8['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [30]:
min(df_edf8['Percent Positive']), max(df_edf8['Percent Positive']), df_edf8['Total Brightness per Signal Area'].min(), df_edf8['Total Brightness per Signal Area'].max()

(0.09111326167594314,
 36.610482686171586,
 0.05926339370399347,
 0.5542153680797345)

In [31]:
inds = (df_reduced['Dose'] == 1e9)

df_edf8_2 = df_reduced.loc[inds]

In [32]:
# Next, we can reduce this to just to round 7.
inds = ((df_edf8_2['Virus'] == '9P31') | (df_edf8_2['Virus'] == '9P36')) & ((df_edf8_2['Round'] > 6) & (df_edf8_2['Round'] < 10))

df_edf8_2 = df_edf8_2.loc[inds]

In [33]:
# We can determine the mean of each of the values
df_edf8_2 = df_edf8_2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\805570888.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_edf8_2 = df_edf8_2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [34]:
source = ColumnDataSource(df_edf8_2)

p = figure(x_range = ['9P31',
                      '9P36'],
          y_range = ['Tek',
                     'Sclo1c1',
                     'Slco1a4',
                     'Slc7a1',
                     'Slc6a6',
                     'Slc2a1', 
                     'Slc22a8',
                     'Serinc3',
                     'Prom1',
                     'Podxl',
                     'Pecam1', 
                     'Paqr5', 
                     'Ocln', 
                     'Ly6e', 
                     'Ly6c1', 
                     'Ly6a', 
                     'Kitl', 
                     'Kdr', 
                     'Itm2b', 
                     'Igf1r', 
                     'Ifitm3', 
                     'Ifitm2', 
                     'Flt1', 
                     'Fcgrt', 
                     'Esam', 
                     'Eng',
                     'Clic4', 
                     'Clic1', 
                     'Clec2d', 
                     'Cldn5', 
                     'Cdh5', 
                     'Cd34', 
                     'Car4', 
                     'Bsg', 
                     'App', 
                     'Adgrl4',
                     'Adgrf5', 
                     'Acvrl1', 
                     'Abcg2', 
                     'Abcb1a'],
           title = 'Candidate Receptors',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=300, 
           height=1000)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf8['Total Brightness per Signal Area'].min(), high = df_edf8['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [35]:
inds = (df_edf8_2['Virus'] != 'PHP.C1') & (df_edf8_2['Virus'] != 'PHP.C3')
min(df_edf8_2.loc[inds]['Percent Positive']), max(df_edf8_2.loc[inds]['Percent Positive']), min(df_edf8_2.loc[inds]['Total Brightness per Signal Area']), max(df_edf8_2.loc[inds]['Total Brightness per Signal Area'])

(3.030669586433144, 55.27691900821151, 0.11078512033503711, 0.6136580304390394)

## Extended Data Figure 10

In [36]:
# Select the round for the corresponding figure
inds = (df_reduced['Round'] == 11) & (df_reduced['Dose'] != 5e8)

# Make this a new dataframe df_edf10
df_edf10 = df_reduced.loc[inds]

In [37]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_edf10 = df_edf10.astype({"Dose": str})

In [38]:
# We can determine the mean of each of the values
df_edf10 = df_edf10.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\293955527.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_edf10 = df_edf10.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [39]:
df_edf10

,Virus,Receptor,Dose,Round,Plate,Well,Count,Bright Field Area,Signal Area,Total Brightness,DMSO (%),Brinzolamide (nM),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,9P31,Ly6c1,100000000.0,11.0,3.0,95.0,1216.666667,131693.666667,62121.000000,41035.930719,0.0,0.0,47.392678,23.696339,0.659274
1,9P31,Ly6c1,1000000000.0,11.0,1.0,92.0,1870.000000,164873.333333,101593.333333,76422.390850,0.0,0.0,61.617925,30.808962,0.751415
2,9P31,Ly6c1,50000000.0,11.0,3.0,92.0,842.333333,132767.666667,33681.666667,16502.526797,0.0,0.0,25.456869,12.728435,0.489483
3,9P31,None,100000000.0,11.0,4.0,95.0,748.666667,176497.333333,18322.333333,4736.210458,0.0,0.0,10.348949,5.174475,0.254192
4,9P31,None,1000000000.0,11.0,2.0,92.0,1694.666667,190177.666667,40022.333333,19428.464052,0.0,0.0,21.128955,10.564477,0.482137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,eC 9,Ly6c1,1000000000.0,11.0,1.0,32.0,2074.000000,173347.333333,119450.333333,89862.394771,0.0,0.0,68.909609,34.454805,0.752369
92,eC 9,Ly6c1,50000000.0,11.0,3.0,35.0,971.000000,135723.000000,32104.000000,13861.138562,0.0,0.0,24.215653,12.107827,0.431993
93,eC 9,None,100000000.0,11.0,4.0,32.0,970.000000,166380.333333,22176.666667,6433.146405,0.0,0.0,13.313465,6.656733,0.288055
94,eC 9,None,1000000000.0,11.0,2.0,32.0,2404.333333,160288.333333,95579.333333,59451.560784,0.0,0.0,59.938571,29.969286,0.622212


In [40]:
for virus in ['PHP.eB', 'PHP.C2', 'AAV-F', 'eC 1', 'eC 3', 'eC 4', 'eC 5', 'eC 8','eC 9', 'eC 11', 'eC 12', 'eC 14', 'eC 15', 'eC 16', 'eC 19']:
    inds = (df_edf10['Virus'] == virus)
    
    source = ColumnDataSource(df_edf10[inds])

    p = figure(x_range = ['None',
                         'Ly6c1'],
              y_range = ['50000000.0',
                         '100000000.0',
                         '1000000000.0'],
               title = virus,
               x_axis_label = 'Receptor',
               y_axis_label = 'Dose',
               width=400, 
               height=400)

    color_mapper = LinearColorMapper(palette = Viridis256, low = df_edf10['Total Brightness per Signal Area'].min(), high = df_edf10['Total Brightness per Signal Area'].max())

    color_bar = ColorBar(color_mapper = color_mapper,
                         location = (0, 0),
                         ticker = BasicTicker())

    p.add_layout(color_bar, 'right')

    p.scatter(x = 'Receptor', y = 'Dose', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

    p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

    p.xaxis.major_label_orientation = np.pi/2

    p.output_backend = "svg"

    show(p)

In [41]:
min(df_edf10['Percent Positive']), max(df_edf10['Percent Positive']), min(df_edf10['Total Brightness per Signal Area']), max(df_edf10['Total Brightness per Signal Area'])

(2.380977081028067, 93.44158123130579, 0.13414699322463494, 0.8121948225959175)

## Figure 4

### Ly6a mutated

In [42]:
inds = (df_reduced['Virus'] == 'PHP.eB') & (df_reduced['Round'] == 12) & (df_reduced['Dose'] != 1e10) 

df_4_pt1 = df_reduced.loc[inds]

In [43]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_4_pt1 = df_4_pt1.astype({"Dose": str})

In [44]:
# We can determine the mean of each of the values
df_4_pt1 = df_4_pt1.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\3973557576.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_4_pt1 = df_4_pt1.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [45]:
df_4_pt1

,Virus,Receptor,Dose,Round,Plate,Well,Count,Bright Field Area,Signal Area,Total Brightness,DMSO (%),Brinzolamide (nM),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,PHP.eB,FAM234A,10000000.0,12.0,3.0,59.0,357.333333,195148.000000,6195.666667,1047.418301,0.0,0.0,3.175997,1.587998,0.171002
1,PHP.eB,FAM234A,100000000.0,12.0,3.0,56.0,402.333333,174411.666667,9501.666667,2177.211765,0.0,0.0,5.477180,2.738590,0.228673
2,PHP.eB,FAM234A,1000000000.0,12.0,3.0,53.0,1808.666667,195344.666667,35656.333333,16211.035294,0.0,0.0,18.641805,9.320903,0.452504
3,PHP.eB,Ly6a,10000000.0,12.0,1.0,11.0,395.666667,180150.666667,12195.666667,3880.766013,0.0,0.0,6.759724,3.379862,0.316858
4,PHP.eB,Ly6a,100000000.0,12.0,1.0,8.0,624.333333,173581.666667,20636.666667,7906.424837,0.0,0.0,11.906653,5.953326,0.381732
5,PHP.eB,Ly6a,1000000000.0,12.0,1.0,5.0,1773.000000,197005.666667,56629.666667,34600.626144,0.0,0.0,28.697016,14.348508,0.608236
6,PHP.eB,Ly6a A58R,10000000.0,12.0,1.0,35.0,353.666667,199479.000000,5888.333333,910.546405,0.0,0.0,2.944195,1.472098,0.158178
7,PHP.eB,Ly6a A58R,100000000.0,12.0,1.0,32.0,456.333333,187567.666667,8831.666667,1711.758170,0.0,0.0,4.711779,2.355889,0.193027
8,PHP.eB,Ly6a A58R,1000000000.0,12.0,1.0,29.0,1686.666667,202095.333333,32396.333333,13972.865359,0.0,0.0,16.048569,8.024285,0.430672
9,PHP.eB,None,10000000.0,12.0,6.0,11.0,324.333333,207820.333333,7265.333333,1597.079739,0.0,0.0,3.431204,1.715602,0.195759


In [46]:
source = ColumnDataSource(df_4_pt1)

p = figure(x_range = ['10000000.0',
                      '100000000.0',
                      '1000000000.0'],
          y_range = ['Ly6a A58R',
                     'Ly6a'],
           title = 'Mutated Ly6a Infectivity Assay',
           x_axis_label = 'Dose',
           y_axis_label = 'Receptor',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt1['Total Brightness per Signal Area'].min(), high = df_4_pt1['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Dose', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [47]:
min(df_4_pt1['Percent Positive']), max(df_4_pt1['Percent Positive']), min(df_4_pt1['Total Brightness per Signal Area']), max(df_4_pt1['Total Brightness per Signal Area'])

(2.9441953260791274,
 28.697016215018266,
 0.15817774507779078,
 0.6082360803453385)

In [48]:
inds = (df_reduced['Virus'] == '9P31') & (df_reduced['Round'] == 12) & (df_reduced['Dose'] != 1e10) 

df_4_pt2 = df_reduced.loc[inds]

In [49]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_4_pt2 = df_4_pt2.astype({"Dose": str})

In [50]:
# We can determine the mean of each of the values
df_4_pt2 = df_4_pt2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\290421689.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_4_pt2 = df_4_pt2.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [51]:
df_4_pt2

,Virus,Receptor,Dose,Round,Plate,Well,Count,Bright Field Area,Signal Area,Total Brightness,DMSO (%),Brinzolamide (nM),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,9P31,None,10000000.0,12.0,6.0,23.0,352.666667,210319.666667,6926.666667,1192.504575,0.0,0.0,3.273152,1.636576,0.172897
1,9P31,None,100000000.0,12.0,6.0,20.0,492.333333,185020.666667,9506.666667,1923.607843,0.0,0.0,5.152370,2.576185,0.202365
2,9P31,None,1000000000.0,12.0,6.0,17.0,1831.666667,219790.000000,32096.000000,13574.368627,0.0,0.0,14.669777,7.334888,0.421575
3,9P31,TM 118,10000000.0,12.0,2.0,35.0,505.000000,169801.000000,11429.000000,2944.920261,0.0,0.0,6.798047,3.399023,0.258020
4,9P31,TM 118,100000000.0,12.0,2.0,32.0,645.333333,175675.333333,17521.000000,5501.328105,0.0,0.0,10.004478,5.002239,0.294237
5,9P31,TM 118,1000000000.0,12.0,2.0,29.0,1905.666667,176629.333333,59149.333333,35155.168627,0.0,0.0,33.979604,16.989802,0.592559
6,9P31,TM 119,10000000.0,12.0,2.0,59.0,357.333333,181537.333333,6316.666667,1148.273203,0.0,0.0,3.537205,1.768603,0.182354
7,9P31,TM 119,100000000.0,12.0,2.0,56.0,394.666667,182717.333333,7534.000000,1542.190850,0.0,0.0,4.105378,2.052689,0.201007
8,9P31,TM 119,1000000000.0,12.0,2.0,53.0,1681.333333,184832.666667,35939.333333,16731.056209,0.0,0.0,19.479200,9.739600,0.462970
9,9P31,TM 120,10000000.0,12.0,2.0,83.0,355.333333,176487.333333,6475.666667,1319.573856,0.0,0.0,3.724741,1.862370,0.201261


In [52]:
source = ColumnDataSource(df_4_pt2)

p = figure(x_range = ['10000000.0',
                      '100000000.0',
                      '1000000000.0'],
          y_range = ['msCar4 L71R',
                     'msCar4'],
           title = 'Mutated Car4 Infectivity Assay',
           x_axis_label = 'Dose',
           y_axis_label = 'Receptor',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt2['Total Brightness per Signal Area'].min(), high = df_4_pt2['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Dose', y = 'Receptor', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [53]:
min(df_4_pt2['Percent Positive']), max(df_4_pt2['Percent Positive']), min(df_4_pt2['Total Brightness per Signal Area']), max(df_4_pt2['Total Brightness per Signal Area'])

(3.2731518082256064,
 99.19942270672327,
 0.14890550513904957,
 0.8574769509049874)

In [54]:
inds = (df_reduced['Dose'] == 1e9) & (df_reduced['Round'] == 10)

df_4_pt3 = df_reduced.loc[inds]

In [55]:
inds = (df_4_pt3['Virus'] == '9P31') | (df_4_pt3['Virus'] == '9P36')

df_4_pt3 = df_4_pt3.loc[inds]

In [56]:
inds = (df_4_pt3['Receptor'] == 'Car4') | (df_4_pt3['Receptor'] == 'rsCar4') | (df_4_pt3['Receptor'] == 'hsCar4')

df_4_pt3 = df_4_pt3.loc[inds]

In [57]:
# We can determine the mean of each of the values
df_4_pt3 = df_4_pt3.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\503976149.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_4_pt3 = df_4_pt3.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [58]:
source = ColumnDataSource(df_4_pt3)

p = figure(x_range = ['Car4',
                     'rsCar4',
                     'hsCar4'],
          y_range = ['9P36',
                     '9P31'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt3['Total Brightness per Signal Area'].min(), high = df_4_pt3['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [59]:
min(df_4_pt3['Percent Positive']), max(df_4_pt3['Percent Positive']), min(df_4_pt3['Total Brightness per Signal Area']), max(df_4_pt3['Total Brightness per Signal Area'])

(4.907121727458143, 51.98639846511173, 0.4571333572915233, 0.7826969799755489)

In [60]:
inds = (df_reduced['Round'] == 12)

df_4_pt4 = df_reduced.loc[inds]

In [61]:
inds = (df_4_pt4['Virus'] == '9P31') | (df_4_pt4['Virus'] == '9P36')

df_4_pt4 = df_4_pt4.loc[inds]

In [62]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_4_pt4 = df_4_pt4.astype({"Dose": str})

In [63]:
# We can determine the mean of each of the values
df_4_pt4 = df_4_pt4.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\153271839.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_4_pt4 = df_4_pt4.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [64]:
df_4_pt4

,Virus,Receptor,Dose,Round,Plate,Well,Count,Bright Field Area,Signal Area,Total Brightness,DMSO (%),Brinzolamide (nM),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,9P31,None,10000000.0,12.0,6.0,23.0,352.666667,210319.666667,6926.666667,1192.504575,0.0,0.0,3.273152,1.636576,0.172897
1,9P31,None,100000000.0,12.0,6.0,20.0,492.333333,185020.666667,9506.666667,1923.607843,0.0,0.0,5.152370,2.576185,0.202365
2,9P31,None,1000000000.0,12.0,6.0,17.0,1831.666667,219790.000000,32096.000000,13574.368627,0.0,0.0,14.669777,7.334888,0.421575
3,9P31,None,10000000000.0,12.0,6.0,14.0,2790.000000,199735.333333,74485.666667,53158.105882,0.0,0.0,37.218069,18.609034,0.705140
4,9P31,TM 118,10000000.0,12.0,2.0,35.0,505.000000,169801.000000,11429.000000,2944.920261,0.0,0.0,6.798047,3.399023,0.258020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,9P36,msCar4,10000000000.0,12.0,1.0,50.0,1421.666667,196121.666667,224360.666667,198932.362092,0.0,0.0,114.546862,57.273431,0.886355
60,9P36,msCar4 L71R,10000000.0,12.0,1.0,83.0,990.000000,207278.000000,26323.333333,9627.698039,0.0,0.0,12.539238,6.269619,0.346492
61,9P36,msCar4 L71R,100000000.0,12.0,1.0,80.0,2093.666667,183489.333333,77427.333333,49314.413072,0.0,0.0,43.148922,21.574461,0.631042
62,9P36,msCar4 L71R,1000000000.0,12.0,1.0,77.0,2321.333333,221442.000000,200059.666667,177895.938562,0.0,0.0,91.060058,45.530029,0.889192


In [65]:
for receptor in ['msCar4', 'huCar4', 'TM 118', 'TM 119', 'TM 120', 'TM 121']:
    inds = (df_4_pt4['Receptor'] == receptor)
    
    source = ColumnDataSource(df_4_pt4[inds])

    p = figure(x_range = ['9P31',
                          '9P36'],
              y_range = ['10000000.0',
                         '100000000.0',
                         '1000000000.0',
                         '10000000000.0'],
               title = receptor,
               x_axis_label = 'Virus',
               y_axis_label = 'Dose',
               width=400, 
               height=400)

    color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt4['Total Brightness per Signal Area'].min(), high = df_4_pt4['Total Brightness per Signal Area'].max())

    color_bar = ColorBar(color_mapper = color_mapper,
                         location = (0, 0),
                         ticker = BasicTicker())

    p.add_layout(color_bar, 'right')

    p.scatter(x = 'Virus', y = 'Dose', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

    p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

    p.xaxis.major_label_orientation = np.pi/2

    p.output_backend = "svg"

    show(p)

In [66]:
inds = (df_reduced['Dose'] == 1e9) & (df_reduced['Round'] == 12)

df_4_pt4 = df_reduced.loc[inds]

In [67]:
inds = (df_4_pt4['Virus'] == '9P31') | (df_4_pt4['Virus'] == '9P36')

df_4_pt4 = df_4_pt4.loc[inds]

In [68]:
inds = (df_4_pt4['Receptor'] == 'msCar4') | (df_4_pt4['Receptor'] == 'huCar4') | (df_4_pt4['Receptor'] == 'TM 118') | (df_4_pt4['Receptor'] == 'TM 119') | (df_4_pt4['Receptor'] == 'TM 120') | (df_4_pt4['Receptor'] == 'TM 121')
 
df_4_pt4 = df_4_pt4.loc[inds]

In [69]:
# We can determine the mean of each of the values
df_4_pt4 = df_4_pt4.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\153271839.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_4_pt4 = df_4_pt4.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [70]:
source = ColumnDataSource(df_4_pt4)

p = figure(x_range = ['msCar4', 
                      'huCar4', 
                      'TM 118', 
                      'TM 119', 
                      'TM 120', 
                      'TM 121'],
          y_range = ['9P36',
                     '9P31'],
           title = 'Receptor comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=400, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_4_pt4['Total Brightness per Signal Area'].min(), high = df_4_pt4['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

---------

In [71]:
inds = (df_reduced['DMSO (%)'] == 0.3) & (df_reduced['Dose'] == 1e9)

df_x = df_reduced.loc[inds]

In [72]:
inds = df_x['Signal Area'].astype(float) < df_x['Bright Field Area']*1
df_x_tempfix = df_x.loc[inds]

In [73]:
# We can determine the mean of each of the values
df_x_grouped = df_x_tempfix.groupby(['Virus', 'Receptor', 'Dose', 'Brinzolamide (nM)']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\1173430985.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_x_grouped = df_x_tempfix.groupby(['Virus', 'Receptor', 'Dose', 'Brinzolamide (nM)']).mean().reset_index()


In [74]:
df_x_grouped

,Virus,Receptor,Dose,Brinzolamide (nM),Round,Plate,Well,Count,Bright Field Area,Signal Area,Total Brightness,DMSO (%),Percent Positive,Percent Positive Scaled,Total Brightness per Signal Area
0,9P31,Ly6a,1.000000e+09,0.0,14.0,2.000000,62.0,460.333333,165040.333333,8576.000000,1588.126797,0.3,5.253367,2.626684,0.182588
1,9P31,Ly6a,1.000000e+09,1.0,14.0,2.000000,86.0,513.000000,145324.666667,9911.000000,1977.667974,0.3,6.998562,3.499281,0.191448
2,9P31,Ly6a,1.000000e+09,100.0,14.0,2.000000,80.0,565.666667,163442.000000,10001.666667,2108.150327,0.3,6.119740,3.059870,0.210205
3,9P31,Ly6a,1.000000e+09,300.0,14.0,2.000000,71.0,522.666667,163239.666667,9267.000000,1847.569935,0.3,5.836664,2.918332,0.197680
4,9P31,msCar4,1.000000e+09,0.0,14.0,1.000000,44.0,2386.000000,174275.333333,83334.666667,56577.683660,0.3,47.781034,23.890517,0.678192
5,9P31,msCar4,1.000000e+09,1.0,14.0,1.000000,62.0,1640.666667,182369.333333,44402.000000,22011.505882,0.3,24.394487,12.197243,0.495726
6,9P31,msCar4,1.000000e+09,100.0,14.0,1.000000,56.0,468.333333,194957.000000,8140.000000,1607.064052,0.3,4.171395,2.085697,0.194691
7,9P31,msCar4,1.000000e+09,300.0,14.0,1.000000,50.0,550.333333,174530.333333,9253.666667,1727.824837,0.3,5.291137,2.645569,0.184513
8,9P36,Ly6a,1.000000e+09,0.0,14.0,2.666667,2.0,649.000000,118600.666667,15204.000000,4014.737255,0.3,13.954301,6.977150,0.250130
9,9P36,Ly6a,1.000000e+09,1.0,14.0,3.000000,20.0,823.000000,172456.333333,16134.666667,3790.577778,0.3,9.360123,4.680061,0.234900


In [75]:
# We need to convert the dose values to a string to allow us to plot catagorically
df_x_grouped = df_x_grouped.astype({"Brinzolamide (nM)": str})
df_x_grouped = df_x_grouped.astype({"Dose": str})

In [76]:
for receptor in ['msCar4', 'Ly6a']:
    inds = (df_x_grouped['Receptor'] == receptor)

    source = ColumnDataSource(df_x_grouped[inds])

    p = figure(x_range = ['0.0',
                         '1.0',
                         '100.0',
                         '300.0'],
              y_range = ['9P36',
                         '9P31',
                         'PHP.eB'],
               title = receptor,
               x_axis_label = 'Dose',
               y_axis_label = 'Virus',
               width=300, 
               height=400)

    color_mapper = LinearColorMapper(palette = Viridis256, low = df_x_grouped[ 'Total Brightness per Signal Area'].min(), high = df_x_grouped['Total Brightness per Signal Area'].max())

    color_bar = ColorBar(color_mapper = color_mapper,
                         location = (0, 0),
                         ticker = BasicTicker())

    p.add_layout(color_bar, 'right')

    p.scatter(x = 'Brinzolamide (nM)', y = 'Virus', size = 'Percent Positive Scaled', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

    p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

    p.xaxis.major_label_orientation = np.pi/2

    p.output_backend = "svg"

    show(p)

In [77]:
min(df_x_grouped['Percent Positive']), max(df_x_grouped['Percent Positive']), min(df_x_grouped['Total Brightness per Signal Area']), max(df_x_grouped['Total Brightness per Signal Area'])

(3.3721833576467795, 63.25989434041124, 0.12219945808508252, 0.75348929569693)

----------

In [78]:
inds = (df_reduced['Round'] == 15) & (df_reduced['Dose'] == 1e9)
df_y = df_reduced.loc[inds]

In [79]:
# We can determine the mean of each of the values
df_y_grouped = df_y.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\2550513952.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_y_grouped = df_y.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [80]:
inds = (df_y_grouped['Virus'] == 'TM126') | (df_y_grouped['Virus'] == 'TM127')

df_y_grouped = df_y_grouped.loc[inds]

In [81]:
source = ColumnDataSource(df_y_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'rhCar4',
                      'huCar4'],
          y_range = ['TM127',
                     'TM126'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=300, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_y_grouped['Total Brightness per Signal Area'].min(), high = df_y_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [82]:
# We can determine the mean of each of the values
df_y_grouped = df_y.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\2550513952.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_y_grouped = df_y.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [83]:
inds = (df_y_grouped['Virus'] == 'RR045') | (df_y_grouped['Virus'] == 'RR475') | \
(df_y_grouped['Virus'] == 'RD4') | (df_y_grouped['Virus'] == 'RDC16') | (df_y_grouped['Virus'] == 'MP3') | (df_y_grouped['Virus'] == 'RDM18') | \
(df_y_grouped['Virus'] == 'MP1A') | (df_y_grouped['Virus'] == 'RDM5') | (df_y_grouped['Virus'] == 'RD2') | (df_y_grouped['Virus'] == 'RDC9') | \
(df_y_grouped['Virus'] == 'RM174') | (df_y_grouped['Virus'] == 'RD5') | (df_y_grouped['Virus'] == 'MR2') | (df_y_grouped['Virus'] == 'RDC8')| \
(df_y_grouped['Virus'] == 'MP13') | (df_y_grouped['Virus'] == 'RM222')

df_y_grouped = df_y_grouped.loc[inds]

In [84]:
source = ColumnDataSource(df_y_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'rhCar4',
                      'huCar4'],
          y_range = ['RR045',
                     'RR475',
                     'RD4',
                     'RDC16',
                     'MP3',
                     'RDM18',
                     'MP1A',
                     'RDM5',
                     'RD2',
                     'RDC9',
                     'RM174',
                     'RD5',
                     'MR2',
                     'RDC8',
                     'MP13',
                     'RM222'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=300, 
           height=700)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_y_grouped['Total Brightness per Signal Area'].min(), high = df_y_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

----------------

In [85]:
# We want to make a new dataframe with just the 1e9 dose.
# First, we can find the indices of these receptors.
inds = (df_reduced['Round'] == 13) & (df_reduced['Dose'] == 1e9)

# Make this a new dataframe df_edf5
df_3c = df_reduced.loc[inds]

In [86]:
# We can determine the mean of each of the values
df_3c = df_3c.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\255182437.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_3c = df_3c.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [87]:
source = ColumnDataSource(df_3c)

p = figure(x_range = ['PHP.eB',
                      'CAP-B10',
                      'CAP-B22',
                      '9P08',
                      '9P16',
                      'PHP.N',
                      'PHP.C1',
                      'PHP.C2',
                      'PHP.C3',
                      'PHP.C4',
                      'AAV-F',
                      '9P13',
                      '9P33',
                      '9P39'],
          y_range = ['CD59',
                     'Ly6S',
                     'Lynx1',
                     'Lypd5',
                     'Ly6h',
                     'Ly6e', 
                     'Ly6c1',
                     'Ly6a'],
           title = 'Receptor screen of Ly6 family',
           y_axis_label = 'Receptor',
           x_axis_label = 'Virus',
           width=800, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_3c['Total Brightness per Signal Area'].min(), high = df_3c['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Virus', y = 'Receptor', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [88]:
min(df_3c['Percent Positive']), max(df_3c['Percent Positive']), min(df_3c['Total Brightness per Signal Area']), max(df_3c['Total Brightness per Signal Area'])

(2.543252358210863,
 50.537629310783096,
 0.12910872665751924,
 0.7501429166333894)

--------------

In [89]:
inds = (df_reduced['Round'] == 16) & (df_reduced['Dose'] == 1e9)
df_z = df_reduced.loc[inds]

In [90]:
# We can determine the mean of each of the values
df_z_grouped = df_z.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\544337510.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_z_grouped = df_z.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [91]:
df_z_grouped['Virus'].unique()

array(['9P31', '9P36', 'PHP.eB', 'TM126', 'TM127'], dtype=object)

In [92]:
source = ColumnDataSource(df_z_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      'rhCar4',
                      'huCar4'],
          y_range = ['TM127', 'TM126', '9P36', '9P31', 'PHP.eB'],
           title = 'Car4 comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=300, 
           height=800)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_z_grouped['Total Brightness per Signal Area'].min(), high = df_z_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

#p.output_backend = "svg"

show(p)

In [93]:
min(df_z_grouped['Percent Positive']), max(df_z_grouped['Percent Positive']), min(df_z_grouped['Total Brightness per Signal Area']), max(df_z_grouped['Total Brightness per Signal Area'])

(6.224010290324046, 57.14685919050171, 0.2127868109105774, 0.7992698196288665)

-----------------

In [94]:
inds = (df_reduced['Round'] == 17) & (df_reduced['Dose'] == 1e9) & (df_reduced['Virus'].isin(['PHP.eB', 'CAP-B10', 'CAP-B22']))
df_supp3c = df_reduced.loc[inds]

In [95]:
# We can determine the mean of each of the values
df_supp3c_grouped = df_supp3c.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\826594836.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_supp3c_grouped = df_supp3c.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [96]:
source = ColumnDataSource(df_supp3c_grouped)

p = figure(x_range = ['None',
                      'Ly6a',
                      'Marmoset Ly6E',
                      'Marmoset LyPD5',
                      'Marmoset Lynx1',
                      'Marmoset CD59'],
          y_range = ['CAP-B22','CAP-B10', 'PHP.eB'],
           title = 'Marmoset Receptor Comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=400, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_supp3c_grouped['Total Brightness per Signal Area'].min(), high = df_supp3c_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [97]:
min(df_supp3c_grouped['Percent Positive']), max(df_supp3c_grouped['Percent Positive']), min(df_supp3c_grouped['Total Brightness per Signal Area']), max(df_supp3c_grouped['Total Brightness per Signal Area'])

(4.571867651746359, 52.46005647073576, 0.169256281610034, 0.6864039303681215)

In [98]:
inds = (df_reduced['Round'] == 17) & (df_reduced['Dose'] == 1e9) & (df_reduced['Virus'].isin(['9P31', '9P36'])) & (df_reduced['Receptor'].isin(['None', 'msCar4', 'msCar4 L71A', 'msCar4 L71E', 'msCar4 L71R']))
df_supp7b = df_reduced.loc[inds]

In [99]:
# We can determine the mean of each of the values
df_supp7b_grouped = df_supp7b.groupby(['Virus', 'Receptor', 'Dose', 'Round']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\3470475991.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_supp7b_grouped = df_supp7b.groupby(['Virus', 'Receptor', 'Dose', 'Round']).mean().reset_index()


In [100]:
source = ColumnDataSource(df_supp7b_grouped)

p = figure(x_range = ['None',
                      'msCar4', 
                      'msCar4 L71A',
                      'msCar4 L71E'],
           y_range = ['9P36',
                      '9P31'],
           title = 'Mouse Car4 Mutants',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=500, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_supp7b_grouped['Total Brightness per Signal Area'].min(), high = df_supp7b_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [101]:
min(df_supp7b_grouped['Percent Positive']), max(df_supp7b_grouped['Percent Positive']), min(df_supp7b_grouped['Total Brightness per Signal Area']), max(df_supp7b_grouped['Total Brightness per Signal Area'])

(4.44725107111247, 70.17850867979818, 0.16769122912045664, 0.7940287848885946)

In [102]:
inds = (df_reduced['Round'] == 12) & (df_reduced['Dose'] == 1e9) & (df_reduced['Virus'].isin(['9P31', '9P36'])) & (df_reduced['Receptor'].isin(['None', 'msCar4', 'msCar4 L71A', 'msCar4 L71E', 'msCar4 L71R']))
df_supp7b = df_reduced.loc[inds]

In [103]:
# We can determine the mean of each of the values
df_supp7b_grouped = df_supp7b.groupby(['Virus', 'Receptor', 'Dose', 'Round']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\3470475991.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_supp7b_grouped = df_supp7b.groupby(['Virus', 'Receptor', 'Dose', 'Round']).mean().reset_index()


In [104]:
source = ColumnDataSource(df_supp7b_grouped)

p = figure(x_range = ['None',
                      'msCar4', 
                      'msCar4 L71R'],
           y_range = ['9P36',
                      '9P31'],
           title = 'Mouse Car4 Mutants',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=500, 
           height=400)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_supp7b_grouped['Total Brightness per Signal Area'].min(), high = df_supp7b_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [105]:
min(df_supp7b_grouped['Percent Positive']), max(df_supp7b_grouped['Percent Positive']), min(df_supp7b_grouped['Total Brightness per Signal Area']), max(df_supp7b_grouped['Total Brightness per Signal Area'])

(11.845386671447061, 99.19942270672327, 0.33225567248345084, 0.889191571688542)

In [106]:
inds = (df_reduced['Round'] == 17) & (df_reduced['Dose'] == 1e9) & (df_reduced['Virus'] == 'PHP.C2') & (df_reduced['Receptor'].isin(['None', 'Ly6c1', 'Ly6c1 E31R', 'Ly6c1 Q56A', 'Ly6c1 S94E', 'Ly6c1 triple mutant']))
df_supp7a = df_reduced.loc[inds]

In [107]:
# We can determine the mean of each of the values
df_supp7a_grouped = df_supp7a.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\899358492.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_supp7a_grouped = df_supp7a.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [108]:
source = ColumnDataSource(df_supp7a_grouped)

p = figure(x_range = ['None',
                      'Ly6c1', 
                      'Ly6c1 E31R', 
                      'Ly6c1 Q56A',
                      'Ly6c1 S94E'],
          y_range = ['PHP.C2'],
           title = 'Ly6c1 Mutants',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=600, 
           height=250)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_supp7a_grouped['Total Brightness per Signal Area'].min(), high = df_supp7a_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [109]:
min(df_supp7a_grouped['Percent Positive']), max(df_supp7a_grouped['Percent Positive']), min(df_supp7a_grouped['Total Brightness per Signal Area']), max(df_supp7a_grouped['Total Brightness per Signal Area'])

(13.472929606232798, 66.64325404574008, 0.3131218386800252, 0.7156638722663825)

In [110]:
# source = ColumnDataSource(df_a_grouped)

# p = figure(x_range = ['None',
#                       'msCar4',
#                       'msCar4 Q63G', 
#                       'rhCar4', 
#                       'rhCar4 G63Q',
#                       'huCar4', 
#                       'huCar4 G63Q'],
#           y_range = ['TTD-008',
#                      'TTD-004',
#                      'TTD-001',
#                      '9P36',
#                      '9P31'],
#            title = 'Car4 species comparison with mutation',
#            x_axis_label = 'Receptor',
#            y_axis_label = 'Virus',
#            width=700, 
#            height=500)

# color_mapper = LinearColorMapper(palette = Viridis256, low = df_a_grouped['Total Brightness per Signal Area'].min(), high = df_a_grouped['Total Brightness per Signal Area'].max())

# color_bar = ColorBar(color_mapper = color_mapper,
#                      location = (0, 0),
#                      ticker = BasicTicker())

# p.add_layout(color_bar, 'right')

# p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

# p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

# p.xaxis.major_label_orientation = np.pi/2

# p.output_backend = "svg"

# show(p)

-------------

In [111]:
inds = (df_reduced['Round'] == 18) & (df_reduced['Dose'] == 1e9)
df_b = df_reduced.loc[inds]

In [112]:
# We can determine the mean of each of the values
df_b_grouped = df_b.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()

C:\Users\dgoertse\AppData\Local\Temp\ipykernel_4976\1639484874.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_b_grouped = df_b.groupby(['Virus', 'Receptor', 'Dose']).mean().reset_index()


In [113]:
source = ColumnDataSource(df_b_grouped)

p = figure(x_range = ['None',
                      'msCar4',
                      #'huCar4',
                      'TM 122',
                      'TM 123'],
          y_range = ['9P36',
                     '9P31'],
           title = 'Car4 species and mutants comparison',
           x_axis_label = 'Receptor',
           y_axis_label = 'Virus',
           width=500, 
           height=300)

color_mapper = LinearColorMapper(palette = Viridis256, low = df_b_grouped['Total Brightness per Signal Area'].min(), high = df_b_grouped['Total Brightness per Signal Area'].max())

color_bar = ColorBar(color_mapper = color_mapper,
                     location = (0, 0),
                     ticker = BasicTicker())

p.add_layout(color_bar, 'right')

p.scatter(x = 'Receptor', y = 'Virus', size = 'Percent Positive', fill_color = transform('Total Brightness per Signal Area', color_mapper), source = source)

p.add_tools(HoverTool(tooltips = [('Round', '@Round'), ('Plate', '@Plate'), ('Well', '@Well'), ('Virus', '@Virus'), ('Receptor', '@Receptor'), ('Dose', '@Dose')]))

p.xaxis.major_label_orientation = np.pi/2

p.output_backend = "svg"

show(p)

In [114]:
min(df_b_grouped['Percent Positive']), max(df_b_grouped['Percent Positive']), min(df_b_grouped['Total Brightness per Signal Area']), max(df_b_grouped['Total Brightness per Signal Area'])

(3.2637142227704525,
 64.68666484706984,
 0.15253916265697828,
 0.7253294582496365)